In [1]:
import xarray as xr
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import pickle
import shapely
import networkx as nx  
import folium
import functools
import pyproj
from osgeo import gdal, ogr

In [2]:
@functools.lru_cache
def load_fis_network(url):
    """load the topological fairway information system network (vaarweginformatie.nl)"""

    # get the data from the url
    resp = requests.get(url)
    # convert to file object
    stream = io.StringIO(resp.text)
    
    # This will take a minute or two
    # Here we convert the network to a networkx object
    G = yaml.load(stream, Loader=yaml.Loader)

    # some brief info
    n_bytes = len(resp.content)
    msg = '''Loaded network from {url} file size {mb:.2f}MB. Network has {n_nodes} nodes and {n_edges} edges.'''
    summary = msg.format(url=url, mb=n_bytes / 1000**2, n_edges=len(G.edges), n_nodes=len(G.nodes))
    logger.info(summary)

    # The topological network contains information about the original geometry. 
    # Let's convert those into python shapely objects for easier use later
    for n in G.nodes:
        G.nodes[n]['geometry'] = shapely.geometry.Point(G.nodes[n]['X'], G.nodes[n]['Y'])
    for e in G.edges:
        edge = G.edges[e]
        edge['geometry'] = shapely.wkt.loads(edge['Wkt'])
        edge['length'] = edge_length(edge)    
    
    return G 

def edge_length(edge):
    """compute the great circle length of an edge
    The network version 0.1 contains the lat/lon distance in a length property. 
    But we need the "great circle" or projected distance. 
    Let's define a function to recompute it.
    """
    
    # get the geometry
    geom = edge['geometry']
    # get lon, lat
    lats, lons = np.array(geom).T
    # this requires pyproj 2.3.0
    distance = geod.line_length(lons, lats)

    return distance

def find_closest_edge(G, point):
    """find the closest edge on the graph from a given point"""
    
    distance = np.full((len(G.edges)), fill_value=np.nan)
    for ii, e in enumerate(G.edges):
        distance[ii] = point.distance(G.edges[e]['Info']['geometry'])
    name_edge = list(G.edges)[np.argmin(distance)]
    distance_edge = np.min(distance)
    
    return name_edge, distance_edge

def find_closest_node(G, point):
    """find the closest node on the graph from a given point"""
    
    distance = np.full((len(G.nodes)), fill_value=np.nan)
    for ii, n in enumerate(G.nodes):
        distance[ii] = point.distance(G.nodes[n]['geometry'])
    name_node = list(G.nodes)[np.argmin(distance)]
    distance_node = np.min(distance)
    
    return name_node, distance_node

## Data on the nodes and OSR-output points

In [3]:
station_data_0 = xr.open_dataset(r'C:\Users\floorbakker\surfdrive\Documents\PhD SALTISolutions\03_Models\OSR2019\OSR data\nc\SDS-NG03_01jan-31mrt_2019SDS-NG03_01jan-31mrt_2019-selected-stations.nc')
station_data_1 = xr.open_dataset(r'C:\Users\floorbakker\surfdrive\Documents\PhD SALTISolutions\03_Models\OSR2019\OSR data\nc\SDS-NG03_01apr-30jun_2019SDS-NG03_01apr-30jun_2019-selected-stations.nc')
station_data_2 = xr.open_dataset(r'C:\Users\floorbakker\surfdrive\Documents\PhD SALTISolutions\03_Models\OSR2019\OSR data\nc\SDS-NG03_01jul-30sep_2019SDS-NG03_01jul-30sep_2019-selected-stations.nc')
station_data_3 = xr.open_dataset(r'C:\Users\floorbakker\surfdrive\Documents\PhD SALTISolutions\03_Models\OSR2019\OSR data\nc\SDS-NG03_01okt-31dec_2019SDS-NG03_01okt-31dec_2019-selected-stations.nc')

C:\Users\floorbakker\Anaconda3\envs\gdal\lib\site-packages\xarray\conventions.py:516: SerializationWarning: variable 'NAMPOL' has multiple fill values {'', b''}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
C:\Users\floorbakker\Anaconda3\envs\gdal\lib\site-packages\xarray\conventions.py:516: SerializationWarning: variable 'NAMPOL' has multiple fill values {'', b''}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
C:\Users\floorbakker\Anaconda3\envs\gdal\lib\site-packages\xarray\conventions.py:516: SerializationWarning: variable 'NAMPOL' has multiple fill values {'', b''}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
C:\Users\floorbakker\Anaconda3\envs\gdal\lib\site-packages\xarray\conventions.py:516: SerializationWarning: variable 'NAMPOL' has multiple fill values {'', b''}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [32]:
station_names = station_data_0['GRO']['NAMPOL']
list_of_station_names = [name.decode("utf-8").replace(" ", "")  for name in station_names.values]
list_of_station_names = [name for name in list_of_station_names if 'Dummy' not in name]
layer_thickness = [0.06,0.18,0.11,0.11,0.11,0.11,0.11,0.09,0.06,0.06]

In [5]:
reference_time = np.datetime64('2019-01-01')
time_data_stations= []
water_level_data_stations = []
for station_data in [station_data_0,station_data_1,station_data_2,station_data_3]:
    time_data_stations.extend([(time-reference_time)/np.timedelta64(1, 's') for time in station_data['GRO']['TIME'].values])

In [6]:
url = 'https://zenodo.org/record/4578289/files/network_digital_twin_v0.2.yaml'

# create a cached version to speed up loading (remove cached file if a better yaml file is available)
fname = "fis_cache\\{}.pkl".format('FIS')

if os.path.exists(fname):
    print('I am loading cached network')
    with open(fname, 'rb') as pkl_file:
        FG = pickle.load(pkl_file)
        pkl_file.close()

else:
    print('I am getting new network')
    FG = load_fis_network(url)

    os.makedirs(os.path.dirname(fname), exist_ok=True)
    with open(fname, 'wb') as pkl_file:
        pickle.dump(FG, pkl_file)
        pkl_file.close()

I am loading cached network


In [7]:
file_path1 = os.path.join(os.getcwd(),'02_hydrodynamic_data','Polygon FIS Network PoR.kml')
file_path1 = os.path.join(os.getcwd(),'02_hydrodynamic_data','Polygon FIS Network PoR.json')

srcDS = gdal.OpenEx(file_path1)
gdal.VectorTranslate('Polygon FIS Network PoR.json', srcDS, format='GeoJSON')

polygon = shapely.geometry.Polygon([[4.047046106288641, 51.994530949601149, 0.0 ], [ 4.041820334307678, 51.985794447379547, 0.0 ], [ 4.032638009410599, 51.987359301430168, 0.0 ], [ 4.026085029365865, 51.986529814588927, 0.0 ], [ 4.023082985071111, 51.985502004661448, 0.0 ], [ 4.00823290473104, 51.988174970713708, 0.0 ], [ 3.998553814899393, 51.987323088053131, 0.0 ], [ 3.987086024543889, 51.983489630873457, 0.0 ], [ 3.980367408521741, 51.978923864007278, 0.0 ], [ 3.972858821044363, 51.972254400764548, 0.0 ], [ 3.964975414140348, 51.962100235646908, 0.0 ], [ 3.965767078274776, 51.958192943818972, 0.0 ], [ 3.9714783688349, 51.946631494630573, 0.0 ], [ 3.983298387670107, 51.927838319363971, 0.0 ], [ 3.987657598176972, 51.919192330532567, 0.0 ], [ 3.996387468717351, 51.916655170259958, 0.0 ], [ 4.001520162387755, 51.917060309257401, 0.0 ], [ 4.012557173047844, 51.920229693122977, 0.0 ], [ 4.048001660981074, 51.929016004063037, 0.0 ], [ 4.074660383723856, 51.934902766449348, 0.0 ], [ 4.093265372959262, 51.934893193914142, 0.0 ], [ 4.109255167927888, 51.93372388247014, 0.0 ], [ 4.126734837924461, 51.932516710383318, 0.0 ], [ 4.139980260895813, 51.931607190411903, 0.0 ], [ 4.147852124552989, 51.929605717664337, 0.0 ], [ 4.154871093926527, 51.92643205114458, 0.0 ], [ 4.164068865894088, 51.922235472642527, 0.0 ], [ 4.168497166059588, 51.919961894548123, 0.0 ], [ 4.173765785457206, 51.917888566713337, 0.0 ], [ 4.177471853896762, 51.916075611730989, 0.0 ], [ 4.19143525340012, 51.910078064370317, 0.0 ], [ 4.202511571167557, 51.904617634287582, 0.0 ], [ 4.212615864448434, 51.899592844692521, 0.0 ], [ 4.219017538516361, 51.895881651452001, 0.0 ], [ 4.223832400198837, 51.890530353414427, 0.0 ], [ 4.226873738466466, 51.887326711590937, 0.0 ], [ 4.228951660169189, 51.883411780650917, 0.0 ], [ 4.230693981376918, 51.879498998972053, 0.0 ], [ 4.23245567320008, 51.876804618361149, 0.0 ], [ 4.235713735314597, 51.874394644882933, 0.0 ], [ 4.240860704028409, 51.871936856471507, 0.0 ], [ 4.246713039932697, 51.869742419056138, 0.0 ], [ 4.256500276768536, 51.866468683159347, 0.0 ], [ 4.263681165876234, 51.864338061762467, 0.0 ], [ 4.270856008579235, 51.864165082808469, 0.0 ], [ 4.277892120705451, 51.86439728390355, 0.0 ], [ 4.286849974208358, 51.8645882221536, 0.0 ], [ 4.292522161762808, 51.864277013800361, 0.0 ], [ 4.299811679854974, 51.864101583593218, 0.0 ], [ 4.311303788739909, 51.864116069612663, 0.0 ], [ 4.329734623050854, 51.864276123508454, 0.0 ], [ 4.331499517754396, 51.864024053521959, 0.0 ], [ 4.333688920564924, 51.862880975070347, 0.0 ], [ 4.336262605418339, 51.859939034098609, 0.0 ], [ 4.34137585560622, 51.854360268676722, 0.0 ], [ 4.351538393640837, 51.847648283939193, 0.0 ], [ 4.362976373927347, 51.843831785091027, 0.0 ], [ 4.370084445558557, 51.841582728647857, 0.0 ], [ 4.380089933091171, 51.840617401653859, 0.0 ], [ 4.391081067036453, 51.839898516477582, 0.0 ], [ 4.399478875843261, 51.840163345822617, 0.0 ], [ 4.408773767123155, 51.841091522022268, 0.0 ], [ 4.416943663050734, 51.840074266710687, 0.0 ], [ 4.423210503951149, 51.838579340029192, 0.0 ], [ 4.427503697165283, 51.836513860483983, 0.0 ], [ 4.425576770167599, 51.834095807664362, 0.0 ], [ 4.429533318178938, 51.832633476567118, 0.0 ], [ 4.430960005176178, 51.833741029897993, 0.0 ], [ 4.434887510360975, 51.833765267256481, 0.0 ], [ 4.43690383938546, 51.83312364379659, 0.0 ], [ 4.444075259200149, 51.831057027288622, 0.0 ], [ 4.455433085635423, 51.829280289923247, 0.0 ], [ 4.46771578321205, 51.829690776121659, 0.0 ], [ 4.478878685926199, 51.830795076878637, 0.0 ], [ 4.488164498900886, 51.832451656061437, 0.0 ], [ 4.497474736200315, 51.832277533876983, 0.0 ], [ 4.504006118328753, 51.831563991137351, 0.0 ], [ 4.512048063063212, 51.829681547431278, 0.0 ], [ 4.52023617846478, 51.827734951843553, 0.0 ], [ 4.527396173112097, 51.827099310549393, 0.0 ], [ 4.533930342872021, 51.828087046218492, 0.0 ], [ 4.54119648874229, 51.829604882537978, 0.0 ], [ 4.546648500960357, 51.829305402579081, 0.0 ], [ 4.552248982461713, 51.827676804936857, 0.0 ], [ 4.554747912727908, 51.824662856298893, 0.0 ], [ 4.557658515089615, 51.818037740135757, 0.0 ], [ 4.562338329955282, 51.812146737080482, 0.0 ], [ 4.567790268643788, 51.808872879868893, 0.0 ], [ 4.575257216434827, 51.80678730057928, 0.0 ], [ 4.58251573514719, 51.805521253772739, 0.0 ], [ 4.590037627731165, 51.805171699165371, 0.0 ], [ 4.597774284387466, 51.804055987532443, 0.0 ], [ 4.604313450900881, 51.802297307784549, 0.0 ], [ 4.613475866941057, 51.800235897440317, 0.0 ], [ 4.617341098435928, 51.798124369718323, 0.0 ], [ 4.618362378336296, 51.796181342994132, 0.0 ], [ 4.621252023873148, 51.790276378816401, 0.0 ], [ 4.62409979956345, 51.782635434416783, 0.0 ], [ 4.623941953031045, 51.77630929086952, 0.0 ], [ 4.623064754206448, 51.763389962802563, 0.0 ], [ 4.624433481891639, 51.757517082463593, 0.0 ], [ 4.627814435262847, 51.75075427110778, 0.0 ], [ 4.627879472881458, 51.743955186361127, 0.0 ], [ 4.626354092832785, 51.736179071181077, 0.0 ], [ 4.623290219595805, 51.726941768179799, 0.0 ], [ 4.619409050652274, 51.721750961942199, 0.0 ], [ 4.616120550381302, 51.716182645862659, 0.0 ], [ 4.609566079180749, 51.709477107202552, 0.0 ], [ 4.605050592100282, 51.705466495857721, 0.0 ], [ 4.600730591394282, 51.702608721228763, 0.0 ], [ 4.593484968687383, 51.700694538430596, 0.0 ], [ 4.578029217511245, 51.698311310158317, 0.0 ], [ 4.566507458151197, 51.695868601503108, 0.0 ], [ 4.556646021680121, 51.694506492441057, 0.0 ], [ 4.544885547002719, 51.692528879578767, 0.0 ], [ 4.546338018922178, 51.687750048814117, 0.0 ], [ 4.567420841100791, 51.69114986797571, 0.0 ], [ 4.568815806261195, 51.688246505711163, 0.0 ], [ 4.574402919128991, 51.673642115923442, 0.0 ], [ 4.582440212968, 51.675045873653353, 0.0 ], [ 4.603759922942611, 51.67909424032014, 0.0 ], [ 4.609000212678733, 51.682787850019203, 0.0 ], [ 4.61322770400086, 51.687589032555543, 0.0 ], [ 4.616346068304478, 51.688505385639402, 0.0 ], [ 4.618177114470288, 51.689083597445311, 0.0 ], [ 4.611503886862787, 51.700539553693048, 0.0 ], [ 4.618547235224733, 51.704758620916408, 0.0 ], [ 4.621188579458506, 51.70912483940171, 0.0 ], [ 4.625278654281832, 51.71853614447808, 0.0 ], [ 4.62895981688983, 51.726591934181187, 0.0 ], [ 4.631015273970891, 51.731345868647637, 0.0 ], [ 4.632270606035429, 51.73633287458577, 0.0 ], [ 4.634070893980669, 51.74383147071385, 0.0 ], [ 4.633874592083471, 51.752192752634713, 0.0 ], [ 4.630948981882912, 51.759574802772647, 0.0 ], [ 4.629120910421564, 51.765486142676011, 0.0 ], [ 4.629532349597874, 51.770814439938789, 0.0 ], [ 4.630894904718586, 51.77947979515762, 0.0 ], [ 4.630205119517427, 51.784593728340461, 0.0 ], [ 4.644155121160485, 51.784738066131723, 0.0 ], [ 4.64675789658717, 51.785521955122, 0.0 ], [ 4.648523080480585, 51.787655536083861, 0.0 ], [ 4.648380883747016, 51.791059131401887, 0.0 ], [ 4.648653816976214, 51.797693391205897, 0.0 ], [ 4.64901104999494, 51.806929426135078, 0.0 ], [ 4.649005053746082, 51.807578765213528, 0.0 ], [ 4.644316669744375, 51.808880483719882, 0.0 ], [ 4.641063113733887, 51.807100885321567, 0.0 ], [ 4.636305758391581, 51.805050274482483, 0.0 ], [ 4.631933508015191, 51.803574293533259, 0.0 ], [ 4.626968120311867, 51.802438675475052, 0.0 ], [ 4.620497311172034, 51.802272999154503, 0.0 ], [ 4.615951679076103, 51.803311401541073, 0.0 ], [ 4.606272307986039, 51.806284851480697, 0.0 ], [ 4.596159971591947, 51.808126778121583, 0.0 ], [ 4.586158031535925, 51.808844780132617, 0.0 ], [ 4.579447886128047, 51.810058867002901, 0.0 ], [ 4.573060189604023, 51.812295490473929, 0.0 ], [ 4.566012519971096, 51.815434547502242, 0.0 ], [ 4.563773566221736, 51.818687634757516, 0.0 ], [ 4.563201599006845, 51.821957279842117, 0.0 ], [ 4.562968089313304, 51.824543181240337, 0.0 ], [ 4.558540613429947, 51.82964006740454, 0.0 ], [ 4.552433393126092, 51.832316314254932, 0.0 ], [ 4.545014088655261, 51.834290234059893, 0.0 ], [ 4.537158070334188, 51.833393779336141, 0.0 ], [ 4.530038207031499, 51.832102384556208, 0.0 ], [ 4.526108059542169, 51.831813208714102, 0.0 ], [ 4.521717619046028, 51.832590425155978, 0.0 ], [ 4.516732521897168, 51.833960245101871, 0.0 ], [ 4.514236527391809, 51.834496902354807, 0.0 ], [ 4.509974165494395, 51.83553112330867, 0.0 ], [ 4.501820011750192, 51.837106988151767, 0.0 ], [ 4.495840125581275, 51.837258622122228, 0.0 ], [ 4.486864901019976, 51.837137748460393, 0.0 ], [ 4.478183135596048, 51.83618389210767, 0.0 ], [ 4.472439264345458, 51.835228495563783, 0.0 ], [ 4.461775016379061, 51.833908716258769, 0.0 ], [ 4.453922237117189, 51.833744056940958, 0.0 ], [ 4.445880993540623, 51.83521472745835, 0.0 ], [ 4.433867160146994, 51.838656519273393, 0.0 ], [ 4.42470273645797, 51.841440135427547, 0.0 ], [ 4.42153152386223, 51.842243381585881, 0.0 ], [ 4.412288631756962, 51.844837057595839, 0.0 ], [ 4.404628276058093, 51.845147332925059, 0.0 ], [ 4.394141940257916, 51.844056513242528, 0.0 ], [ 4.385970084449699, 51.84384058231597, 0.0 ], [ 4.38009310776012, 51.844281463545563, 0.0 ], [ 4.36989301637785, 51.845910018863457, 0.0 ], [ 4.360483758821787, 51.848313888858193, 0.0 ], [ 4.354349626537848, 51.851008172324264, 0.0 ], [ 4.347909504952052, 51.855404628831913, 0.0 ], [ 4.342408539850937, 51.86054970135617, 0.0 ], [ 4.337899232746361, 51.865294765331257, 0.0 ], [ 4.335433216951134, 51.869785988478348, 0.0 ], [ 4.333552618008243, 51.873809653870651, 0.0 ], [ 4.348428925913814, 51.875946272051308, 0.0 ], [ 4.369163929813908, 51.878317553006468, 0.0 ], [ 4.38839016518595, 51.87815521453539, 0.0 ], [ 4.401107220355129, 51.877058298559803, 0.0 ], [ 4.426040664379522, 51.873026806667077, 0.0 ], [ 4.437079597260853, 51.871386012190683, 0.0 ], [ 4.448943520990141, 51.871564030747429, 0.0 ], [ 4.454263046403941, 51.871932213390657, 0.0 ], [ 4.454612554075805, 51.876499560242912, 0.0 ], [ 4.461854063353627, 51.892501464714812, 0.0 ], [ 4.466785298039618, 51.894872117693943, 0.0 ], [ 4.472093734132137, 51.894358029456889, 0.0 ], [ 4.475597745139166, 51.893088291767661, 0.0 ], [ 4.495575011014354, 51.897022475135103, 0.0 ], [ 4.497482873128506, 51.904708775055248, 0.0 ], [ 4.483610341771596, 51.910345930077462, 0.0 ], [ 4.482760758326021, 51.909513132342788, 0.0 ], [ 4.481354888673367, 51.907659452044847, 0.0 ], [ 4.480171615622785, 51.906587418413928, 0.0 ], [ 4.478656007166948, 51.905614334220239, 0.0 ], [ 4.47420728963661, 51.903986716234108, 0.0 ], [ 4.468091888502901, 51.902252150225287, 0.0 ], [ 4.465003985162294, 51.907646878763657, 0.0 ], [ 4.462309529254382, 51.906989997256652, 0.0 ], [ 4.458607445683542, 51.904702185072551, 0.0 ], [ 4.450510265256744, 51.902876957622773, 0.0 ], [ 4.444600706587229, 51.903309513507153, 0.0 ], [ 4.440641527550353, 51.905721770934349, 0.0 ], [ 4.43802672176208, 51.907881366164169, 0.0 ], [ 4.432905776054799, 51.910555872287738, 0.0 ], [ 4.426112874198747, 51.912467834061907, 0.0 ], [ 4.422383819883729, 51.913168929583513, 0.0 ], [ 4.411878223585475, 51.913540129396402, 0.0 ], [ 4.409521680173807, 51.911005537418113, 0.0 ], [ 4.409980373031104, 51.907863380341098, 0.0 ], [ 4.410238903774557, 51.905760620712883, 0.0 ], [ 4.40792824029834, 51.905695231506868, 0.0 ], [ 4.408236980724604, 51.901856095046021, 0.0 ], [ 4.402375508953853, 51.900192780344398, 0.0 ], [ 4.398972492127681, 51.899588847278451, 0.0 ], [ 4.398510798786443, 51.904247434360137, 0.0 ], [ 4.397094621487874, 51.90646667642546, 0.0 ], [ 4.385614359534216, 51.907989085010612, 0.0 ], [ 4.378087964881123, 51.909686675033377, 0.0 ], [ 4.374128281325, 51.908694908241579, 0.0 ], [ 4.367852653269681, 51.908306138351691, 0.0 ], [ 4.363397244884773, 51.909430864627993, 0.0 ], [ 4.356519147274849, 51.906592072056007, 0.0 ], [ 4.349296405899398, 51.904046653826121, 0.0 ], [ 4.348244134194987, 51.903702932010567, 0.0 ], [ 4.349406111299523, 51.899790994706308, 0.0 ], [ 4.344160954128946, 51.898618840978919, 0.0 ], [ 4.335081568788706, 51.896868371347999, 0.0 ], [ 4.326200276202865, 51.896412521739443, 0.0 ], [ 4.32457076936686, 51.900817957716093, 0.0 ], [ 4.319738626071643, 51.900546337822448, 0.0 ], [ 4.307247475566158, 51.900885572136751, 0.0 ], [ 4.276804688291795, 51.907441861806603, 0.0 ], [ 4.260087950487823, 51.913551219808411, 0.0 ], [ 4.25162487299372, 51.91669563964313, 0.0 ], [ 4.248376913383156, 51.91449410685788, 0.0 ], [ 4.245717112143339, 51.915364621818703, 0.0 ], [ 4.243630834376155, 51.916354360761702, 0.0 ], [ 4.235899751919456, 51.921703697987638, 0.0 ], [ 4.228306802315593, 51.928844045846439, 0.0 ], [ 4.215768636252541, 51.935587344495282, 0.0 ], [ 4.199751571089035, 51.941361577730078, 0.0 ], [ 4.187845586291519, 51.946336059832333, 0.0 ], [ 4.174792346056993, 51.952774770268881, 0.0 ], [ 4.160900691414067, 51.960268474450572, 0.0 ], [ 4.143905642848278, 51.96883626854671, 0.0 ], [ 4.128162638085904, 51.9746278367287, 0.0 ], [ 4.125530375394987, 51.975262010141122, 0.0 ], [ 4.120117786508599, 51.977185994794318, 0.0 ], [ 4.116406216069988, 51.978663950477113, 0.0 ], [ 4.10945387129291, 51.981203138086038, 0.0 ], [ 4.093364376007981, 51.984944583465733, 0.0 ], [ 4.047046106288641, 51.994530949601149, 0.0 ] ] )


In [8]:
nodes = []
edges = []

node_names = []

for edge in FG.edges(data = True):
    node_1 = FG.nodes[edge[0]]
    node_2 = FG.nodes[edge[1]]
    
    if node_1["geometry"].within(polygon) or node_2["geometry"].within(polygon):
        nodes.append(node_1)
        nodes.append(node_2)
        node_names.append(edge[0])
        node_names.append(edge[1])
        edges.append(edge)

In [9]:
FG = nx.DiGraph() #FG_new

for i in range(len(nodes)):
    node = nodes[i]
    node_name = node_names[i]
    FG.add_node(node_name, name = str((node['X'], node['Y'])), Position = (node['X'], node['Y']), geometry = node["geometry"])

for edge in edges:
    FG.add_edge(edge[0], edge[1], Info = edge[2]) 

for i in range(3):
    FG.remove_edge(find_closest_edge(FG,shapely.geometry.Point([4.488648,51.908018]))[0][0],find_closest_edge(FG,shapely.geometry.Point([4.488648,51.908018]))[0][1])

nodes_to_be_removed = ['8860852','B4705_A','B4705_B','B45486_B','L4199_A','B27780191_A','B27780191_B','8867741','B23888_B','B23888_A','L6951_B','L6951_A','L43910_B','L43910_A','B16957_B','B57921_A','L57066_B','L57066_A']
for n in nodes_to_be_removed:
    FG.remove_node(n)

#Node 1
node_name = '8866969B'
node["geometry"] = shapely.geometry.Point([FG.nodes['8866969']['geometry'].x,
                                           FG.nodes['8866969']['geometry'].y])
node['X'] = node["geometry"].x
node['Y'] = node["geometry"].y
FG.add_node(node_name, name = str((node['X'], node['Y'])), Position = (node['X'], node['Y']), geometry = node["geometry"])

Info = FG.edges[('8862214', '8864217')]['Info']
Info['StartJunctionId'] = '8866969'
Info['EndJunctionId'] = '8866969B'
Info['Length'] = 0
Info['length'] = pyproj.Geod(ellps="WGS84").inv(FG.nodes['8866969']['geometry'].x,
                                                FG.nodes['8866969']['geometry'].y,
                                                FG.nodes['8866969B']['geometry'].x,
                                                FG.nodes['8866969B']['geometry'].y,)[2]
Info['Wkt'] = ('LINESTRING (' + str(FG.nodes['8866969']['geometry'].x) + ', ' +
                                str(FG.nodes['8866969']['geometry'].y) + ', ' +
                                str(FG.nodes['8866969B']['geometry'].x) + ', ' +
                                str(FG.nodes['8866969B']['geometry'].y)) + ')'

FG.add_edge('8866969', '8866969B',Info=Info)


#Node 2
node_name = '8866969A'
node["geometry"] = shapely.geometry.Point([FG.nodes['8866969']['geometry'].x,
                                           FG.nodes['8866969']['geometry'].y])
node['X'] = node["geometry"].x
node['Y'] = node["geometry"].y
FG.add_node(node_name, name = str((node['X'], node['Y'])), Position = (node['X'], node['Y']), geometry = node["geometry"])

Info = FG.edges[('8862214', '8864217')]['Info']
Info['StartJunctionId'] = '8866969B'
Info['EndJunctionId'] = '8866969A'
Info['Length'] = 0
Info['length'] = pyproj.Geod(ellps="WGS84").inv(FG.nodes['8866969B']['geometry'].x,
                                                FG.nodes['8866969B']['geometry'].y,
                                                FG.nodes['8866969A']['geometry'].x,
                                                FG.nodes['8866969A']['geometry'].y,)[2]
Info['Wkt'] = ('LINESTRING (' + str(FG.nodes['8866969B']['geometry'].x) + ', ' +
                                str(FG.nodes['8866969B']['geometry'].y) + ', ' +
                                str(FG.nodes['8866969A']['geometry'].x) + ', ' +
                                str(FG.nodes['8866969A']['geometry'].y)) + ')'

FG.add_edge('8866969B', '8866969A',Info=Info)

#Node 3
node_name = '8866969C'
node["geometry"] = shapely.geometry.Point([FG.nodes['8866969']['geometry'].x,
                                           FG.nodes['8866969']['geometry'].y])
node['X'] = node["geometry"].x
node['Y'] = node["geometry"].y
FG.add_node(node_name, name = str((node['X'], node['Y'])), Position = (node['X'], node['Y']), geometry = node["geometry"])

Info = FG.edges[('8862214', '8864217')]['Info']
Info['StartJunctionId'] = '8866969B'
Info['EndJunctionId'] = '8866969C'
Info['Length'] = 0
Info['length'] = pyproj.Geod(ellps="WGS84").inv(FG.nodes['8866969B']['geometry'].x,
                                                FG.nodes['8866969B']['geometry'].y,
                                                FG.nodes['8866969C']['geometry'].x,
                                                FG.nodes['8866969C']['geometry'].y,)[2]
Info['Wkt'] = ('LINESTRING (' + str(FG.nodes['8866969B']['geometry'].x) + ', ' +
                                str(FG.nodes['8866969B']['geometry'].y) + ', ' +
                                str(FG.nodes['8866969C']['geometry'].x) + ', ' +
                                str(FG.nodes['8866969C']['geometry'].y)) + ')'

FG.add_edge('8866969B', '8866969C',Info=Info)

#Recreate simplified Rozenburgse Sluis
Info = FG.edges[('B7951_A','B7951_B')]['Info']
Info['StartJunctionId'] = 'B57361_A'
Info['EndJunctionId'] = 'B7951_B'
Info['length'] = pyproj.Geod(ellps='WGS84').inv(FG.nodes['B7951_A']['geometry'].x,
                                                FG.nodes['B7951_A']['geometry'].y,
                                                FG.nodes['B57361_A']['geometry'].x,
                                                FG.nodes['B57361_A']['geometry'].y)[2]
Info['length'] = 0
Info['Wkt'] = ('LINESTRING (' + str(FG.nodes['8866969B']['geometry'].x) + ', ' +
                                str(FG.nodes['8866969B']['geometry'].y) + ', ' +
                                str(FG.nodes['8866969C']['geometry'].x) + ', ' +
                                str(FG.nodes['8866969C']['geometry'].y)) + ')'

FG.add_edge('B57361_A', 'B7951_B', Info=Info)

In [10]:
FG = FG.to_undirected() 
FG = FG.to_directed()

In [11]:
node_list = []
name_list = []
name_index_list = []
missing_nodes = []
nodes_no_data = ['8862953','B57361_B','8866992','B7951_B','8865732','B14087_A','8866520','8861973','8861547','8861123','8862463','8864025','B57921_B','B44072_B','B44072_A','B44869_A','8866590','8862538','8867387','8867448','B7951_A','B57361_A']
replacement_data_to_nodes = ['8860614','8860614','88614148867008','88614148867008','8861674','8864482','8865942','8861986','8865385','8865385','8865575','8865972','Berth30','S38127','8867784','8867023','8863810','8864653','8864653','88614148867008','88614148867008','88614148867008','8860614']
current_velocity_replacement_data_to_nodes = [0,0,0,0,0,0,'8865942','8861986',0,0,0,0,0,'S38127_A',0,0,0,0,0,0,0,0,0]

for node in enumerate(FG.nodes):
    name_found = False
    for name in enumerate(list_of_station_names):  
        if name[1] == '88651278861067' or name[1] == '88614148867008':
            if name[1] == '88651278861067': subnames = ['8865127','8861067']
            elif name[1] == '88614148867008': subnames = ['8861414','8867008']
            for subname in subnames:
                if subname in node[1]:
                    name_list.append(subname)
                    node_list.append(node[1])
                    name_index_list.append(name[0])
                    name_found = True
                    break   
 
        elif node[1] in nodes_no_data:
            node_index = nodes_no_data.index(node[1])
            name_list.append(replacement_data_to_nodes[node_index])
            node_list.append(node[1])
            name_index_list.append(list_of_station_names.index(replacement_data_to_nodes[node_index]))
            name_found = True
            break
        
        elif type(name[1]) != list:
            if name[1] in node[1]:
                name_list.append(name[1])
                node_list.append(node[1])
                name_index_list.append(name[0])
                name_found = True
                break  
                
    if not name_found:
        missing_nodes.append(node[1])

In [12]:
m = folium.Map(location=[51.85, 4.4], zoom_start = 10, tiles="cartodbpositron")

for node in FG.nodes(data = True):
    points_x = list(node[1]["geometry"].coords.xy[0])
    points_y = list(node[1]["geometry"].coords.xy[1])
    
    point = []
    for i, _ in enumerate(points_x):
        point.append((points_y[i], points_x[i]))
    else:
        folium.Circle(point[0], radius=4,fill=True,fill_opacity=1,tooltip = node[0],popup =node[0]).add_to(m)

for edge in FG.edges(data = True):
    points_x = list(edge[2]["Info"]["geometry"].coords.xy[0])
    points_y = list(edge[2]["Info"]["geometry"].coords.xy[1])
    
    line = []
    for i, _ in enumerate(points_x):
        line.append((points_y[i], points_x[i]))
    
    else:
        popup = folium.Popup(width=500, height=300)
        folium.PolyLine(line, weight = 3, tooltip = [edge[0],edge[1]], popup = [edge[0],edge[1]]).add_to(m)
        
m

In [13]:
water_level_data_stations = []
for index in name_index_list:
    water_level_data_station = []
    for station_data in [station_data_0,station_data_1,station_data_2,station_data_3]:
        data = station_data['ZWL'].transpose('STATION','TIME')
        water_level_data_station.extend(data[index].values)
water_level_data_stations.append(water_level_data_station)

In [16]:
water_level_data = xr.concat([station_data_0['ZWL'], station_data_1['ZWL'], station_data_2['ZWL'], station_data_3['ZWL']], "TIME")
easting_current_velocity_data = xr.concat([station_data_0['ZCURU'], station_data_1['ZCURU'], station_data_2['ZCURU'], station_data_3['ZCURU']], "TIME")
northing_current_velocity_data = xr.concat([station_data_0['ZCURV'], station_data_1['ZCURV'], station_data_2['ZCURV'], station_data_3['ZCURV']], "TIME")
water_level_data = water_level_data.transpose('STATION','TIME')
easting_current_velocity_data = easting_current_velocity_data.transpose('STATIONCUR','LAYER','TIME')
northing_current_velocity_data = northing_current_velocity_data.transpose('STATIONCUR','LAYER','TIME')
layer_data = xr.concat([station_data_0['ZKPOLC'], station_data_1['ZKPOLC'], station_data_2['ZKPOLC'], station_data_3['ZKPOLC']], "TIME")
layer_data = layer_data.transpose('STATION','LAYER','TIME')

water_level_data_nodes = xr.concat([water_level_data[index] for index in name_index_list],'STATION')
layer_data_nodes = xr.concat([-layer_data[index] for index in name_index_list],'STATION')
magnitude_data_nodes = xr.concat([np.sqrt(easting_current_velocity_data[index]**2+northing_current_velocity_data[index]**2) for index in name_index_list],'STATION')
phase_data_nodes = xr.concat([np.degrees(np.arctan2(easting_current_velocity_data[index],northing_current_velocity_data[index])) for index in name_index_list],'STATION')

In [ ]:
hydrodynamic_data = water_level_data_nodes.rename('Water level').to_dataset()
hydrodynamic_data['Layer depth'] = layer_data_nodes
hydrodynamic_data['Current velocity'] = magnitude_data_nodes
hydrodynamic_data['Current direction'] = phase_data_nodes
hydrodynamic_data

In [31]:
station_data_0['ZKPOLC'][0][262]

<xarray.DataArray 'ZKPOLC' (LAYER: 10)>
array([ -0.01982 ,  -2.166119,  -4.22299 ,  -6.190432,  -8.157873, -10.125315,
       -12.092756, -13.88134 , -15.222776, -16.295925], dtype=float32)
Coordinates:
    TIME     datetime64[ns] 2019-01-01
    NAMPOL   object b'Scheurkade                                             ...
Dimensions without coordinates: LAYER
Attributes:
    positive:       up
    standard_name:  positions of the layer-interfaces at constituent stations...
    long_name:      positions of the layer-interfaces at constituent stations...
    units:          m

In [29]:
list_of_station_names.index('Scheurkade')

262